## Team Data

In [3]:
pos = "PointGuard"
t = pos.split("And")[0] if "And" in pos else pos

In [4]:
t

'PointGuard'

In [ ]:
!pip install requests

In [20]:
import requests
import json


## Get completed match

In [21]:
def get_completed_match_id(team_id):
    url = f"https://jonescup.meetagile.com/api/squad-team-squads/{team_id}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        match_id = []
        for match in data['matches']:
            if match['status'] == "COMPLETED":
                match_id.append(match['id'])
        return match_id
    except requests.exceptions.RequestException as e:
        print(f"Error fetching match ID {match_id}: {e}")
        return None

## Fetch all game data

In [22]:
def fetch_all_game_data(team_id, match_ids):
    """
    Fetches stats for each game and stores them individually without accumulating.
    This is the key to solving your problem.
    """
    all_games_data = {}

    for match_id in match_ids:
        try:
            url = f"https://jonescup.meetagile.com/api/match-basketball-stats/{match_id}"
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            game_stats = {
                "team_stats": {},
                "opponent_stats": {},
                "player_stats": {}
            }

            # --- Team & Opponent Stats ---
            for squad in data['squads']:
                stats = squad['stats']['accumulatedStats']
                target = "team_stats" if squad['squadId'] == team_id else "opponent_stats"
                game_stats[target] = stats
                game_stats[target]['squadOffensiveRebounds'] = game_stats[target]['offensiveRebounds'] - game_stats[target]['squadOffensiveRebounds']
                game_stats[target]['squadDefensiveRebounds'] = game_stats[target]['defensiveRebounds'] - game_stats[target]['squadDefensiveRebounds']
               
               
            # --- Player Stats ---
            for roster in data.get('rosters', []):
                if roster.get('squadId') == team_id:
                    roster_id = roster['rosterId']
                    stats = roster['stats']['accumulatedStats']
                    
                    # Only add player if they played
                    if stats.get('timeOnCourt', 0) > 0:
                        game_stats["player_stats"][roster_id] = stats
            
            all_games_data[match_id] = game_stats
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for match {match_id}: {e}")
            continue
            
    return all_games_data

## Team Adv stats

In [23]:
def calculate_team_advanced_stats(data):
    team_stats = dict()
    
    # main team
    team_stats['OffensiveRating'] = data['score'] /data['possessions'] * 100 if data['possessions'] > 0 else 0
    team_stats['DefensiveRating'] = data['Op-score'] / data['Op-possessions'] * 100 if data['Op-possessions'] > 0 else 0
    team_stats['NetRating'] = team_stats['OffensiveRating'] - team_stats['DefensiveRating']
    team_stats['Pace'] = 40 * ((data['possessions'] + data['Op-possessions']) / (2*(data['teamMinutesPlayed'] / 5))) if data['teamMinutesPlayed'] > 0 else 0

    # four factors main team
    team_stats['eFG'] = (data['fieldGoalsMade'] + 0.5 * data['threePointersMade']) / data['fieldGoalsAttempted'] if data['fieldGoalsAttempted'] > 0 else 0
    team_stats['TOV'] = data['turnovers'] / (data['possessions'] + data['squadOffensiveRebounds']) if data['possessions'] > 0 else 0
    team_stats['ORP'] = data['squadOffensiveRebounds'] / (data['squadOffensiveRebounds'] + data['Op-squadDefensiveRebounds']) if (data['squadOffensiveRebounds'] + data['Op-squadDefensiveRebounds']) > 0 else 0
    team_stats['FTR'] = data['freeThrowsAttempted'] / data['fieldGoalsAttempted'] if data['fieldGoalsAttempted'] > 0 else 0

    # four factors opponent team
    team_stats['Op-eFG'] = (data['Op-fieldGoalsMade'] + 0.5 * data['Op-threePointersMade']) / data['Op-fieldGoalsAttempted'] if data['Op-fieldGoalsAttempted'] > 0 else 0
    team_stats['Op-TOV'] = data['Op-turnovers'] / (data['Op-possessions'] + data['Op-squadOffensiveRebounds']) if data['Op-possessions'] > 0 else 0
    team_stats['Op-ORP'] = data['Op-squadOffensiveRebounds'] / (data['Op-squadOffensiveRebounds'] + data['squadDefensiveRebounds']) if (data['Op-squadOffensiveRebounds'] + data['squadDefensiveRebounds']) > 0 else 0
    team_stats['Op-FTR'] = data['Op-freeThrowsAttempted'] / data['Op-fieldGoalsAttempted'] if data['Op-fieldGoalsAttempted'] > 0 else 0

    return team_stats

In [38]:
def get_team_advanced_stats(player_id, all_games_data):
    """
    Calculates team advanced stats by first accumulating totals from all games.
    """
    games_data = all_games_data
    def sum_stat(all_games_data, source_key, stat_key):
        """Helper function to sum a specific stat from the game data."""
        return sum(g[source_key].get(stat_key, 0) for g in all_games_data.values())
    
    # Caluclate the team stats onlt this player id played
    if isinstance(player_id, int):
        player_games = [
            match_id for match_id, game in all_games_data.items()
            if player_id in game.get('player_stats', {})
        ]
        if len(player_games) != len(all_games_data):
            games_data = {k: all_games_data[k] for k in player_games}

    # List of stats that are simple sums
    stat_keys = [
        "fieldGoalsMade", "wonScore", "fieldGoalsAttempted", "threePointersMade",
        "freeThrowsAttempted", "turnovers", "squadOffensiveRebounds",
        "squadDefensiveRebounds", "fastBreakPoints", "pointsOffTurnovers",
        "secondChancePoints", "benchPoints", "largestLead", "largestScoringRun",
        "leadChanges", "pointsInPaint"
    ]
    # --- Build the dictionary cleanly ---
    team_totals = {}

    # Calculate stats for your team
    for key in stat_keys:
        team_totals[key] = sum_stat(games_data, 'team_stats', key)

    # Calculate stats for the opponent
    for key in stat_keys:
        team_totals[f"Op-{key}"] = sum_stat(games_data, 'opponent_stats', key)
    
    # Handle the special cases like totalMinutes separately
    team_totals['teamMinutesPlayed'] = sum_stat(games_data, 'team_stats', 'timeOnCourt') / 60
    team_totals['Op-teamMinutesPlayed'] = sum_stat(games_data, 'opponent_stats', 'timeOnCourt') / 60

    # Calculate Possesion
    team_totals['possessions'] = team_totals['fieldGoalsAttempted'] + team_totals['turnovers'] + 0.44 * team_totals['freeThrowsAttempted'] - team_totals['squadOffensiveRebounds']
    team_totals['Op-possessions'] = team_totals['Op-fieldGoalsAttempted'] + team_totals['Op-turnovers'] + 0.44 * team_totals['Op-freeThrowsAttempted'] - team_totals['Op-squadOffensiveRebounds']
    # Rename 'wonScore' to 'score' for clarity
    team_totals['score'] = team_totals.pop('wonScore')
    team_totals['Op-score'] = team_totals.pop('Op-wonScore')
    
    # Now calculate advanced stats from the accumulated totals
    team_adv_stats = f"Calculating team total stats for player_id : {player_id}"
    if not isinstance(player_id, int):
        team_adv_stats = calculate_team_advanced_stats(team_totals)

    # For now, returning a placeholder
    return {"message": "Team stats calculated here.", "total_games": len(all_games_data), "adv_stats": team_adv_stats, "team_totals": team_totals}

## PLayer adv stats

In [35]:
from collections import Counter

def get_team_stats_for_player_games(player_id, all_games_data):
    """
    Calculates the total team stats from only the games a specific player played.

    This is crucial for providing the correct context (denominator) when
    calculating a player's advanced rate statistics like AST%, USG%, etc.

    Args:
        player_id (str or int): The ID of the player to focus on.
        all_games_data (dict): The main dictionary containing data for all games,
                               keyed by match_id.

    Returns:
        dict: A dictionary of the summed team stats for the relevant games. Returns
              an empty dictionary if the player played in no games.
    """
    # 1. Create a list of games the player participated in.
    player_games = [
        game for game in all_games_data.values()
        if player_id in game.get('player_stats', {})
    ]
    #print(player_games)
    # Return an empty dict if the player has no games to avoid errors.
    if not player_games:
        return {}

    # 2. Use a Counter to efficiently sum the 'team_stats' from those games.
    team_totals_counter = Counter()
    for game in player_games:
        # Safely get the team_stats dictionary for the game and update the counter.
        team_totals_counter.update(game.get('team_stats', {}))
        team_totals_counter.update(game.get('opponent_stats', {}))

    # 3. Convert the Counter object back to a standard dictionary and return it.
    return dict(team_totals_counter)

In [40]:
from collections import Counter

def calculate_player_total_stats(player_id, all_games_data):
    """
    Calculates advanced stats for a single player.
    """
    player_games = {match_id: game for match_id, game in all_games_data.items()
                    if player_id in game['player_stats']}

    if not player_games:
        return {"error": "Player did not play in any completed games."}

    # --- Use the Counter method to get all total stats ---
    total_stats_counter = Counter()
    for game in player_games.values():
        total_stats_counter.update(game['player_stats'][player_id])
    
    # This dictionary now holds all the summed stats
    total_stats = dict(total_stats_counter)
    total_stats['matchCount'] = len(player_games)

    return total_stats # Or return a new dict with the advanced stats

In [41]:
def calculate_player_advanced_stats(roster_id, player_MP, team_totals):
    """
    Calculates advanced stats for a single player.
    """
    player_stats = dict()
    url = f"https://jonescup.meetagile.com/api/roster-division-basketball-stats/{roster_id}"
    response = requests.get(url)
    data = response.json()

    MP = player_MP

    TmTOV = team_totals['turnovers']
    TmMP = team_totals['teamMinutesPlayed']
    TmFGM = team_totals['fieldGoalsMade']
    TmFGA = team_totals['fieldGoalsAttempted']
    TmFTA = team_totals['freeThrowsAttempted']
    
    TmORB = team_totals['squadOffensiveRebounds']
    TmDRB = team_totals['squadDefensiveRebounds']
    TmTRB = TmORB + TmDRB

    OppORB = team_totals['Op-squadOffensiveRebounds']
    OppDRB = team_totals['Op-squadDefensiveRebounds']
    OppTRB = OppORB + OppDRB

    try:
        stats = data[0]['stats']
        matchCount = stats['matchCount']
        average_stats = stats['averageStats']
        percentage_stats = stats['percentageStats']

        player_stats['GP'] = stats['matchCount']
        player_stats['average-points'] = average_stats['score']
        player_stats['average-assists'] = average_stats['assists']
        player_stats['average-rebounds'] = average_stats['rebounds']
        player_stats['average-steals'] = average_stats['steals']
        player_stats['average-blocks'] = average_stats['blocks']
        player_stats['average-turnovers'] = average_stats['turnovers']

        player_stats['3P%'] = percentage_stats['threePointersPercentage'] * 100
        player_stats['FT%'] = percentage_stats['freeThrowsPercentage'] * 100
        player_stats['FG%'] = percentage_stats['fieldGoalsPercentage'] * 100
        player_stats['eFG%'] = percentage_stats['effectiveFieldGoalsPercentage'] * 100
        player_stats['TS%'] = percentage_stats['trueShootingPercentage'] * 100

        player_stats['2PA'] = average_stats['twoPointersAttempted'] * matchCount
        player_stats['2PM'] = average_stats['twoPointersMade'] * matchCount
        player_stats['twoPointersPercentage'] = player_stats['2PM'] / player_stats['2PA'] * 100 if player_stats['2PA'] != 0 else 0

        player_stats['3PA'] = average_stats['threePointersAttempted'] * matchCount
        player_stats['FTA'] = average_stats['freeThrowsAttempted'] * matchCount
        player_stats['AST'] = average_stats['assists'] * matchCount
        player_stats['FGM'] = average_stats['fieldGoalsMade'] * matchCount
        player_stats['FGA'] = average_stats['fieldGoalsAttempted'] * matchCount
        player_stats['TOV'] = average_stats['turnovers'] * matchCount
        player_stats['OREB'] = average_stats['offensiveRebounds'] * matchCount
        player_stats['DREB'] = average_stats['defensiveRebounds'] * matchCount
        player_stats['TREB'] = player_stats['OREB'] + player_stats['DREB']

        allAttempts = (player_stats['2PA'] + player_stats['3PA'] + player_stats['FTA'])
        player_stats['2PA%'] = player_stats['2PA'] / allAttempts * 100 if allAttempts > 0 else 0
        player_stats['3PA%'] = player_stats['3PA'] / allAttempts * 100 if allAttempts > 0 else 0
        player_stats['FTA%'] = player_stats['FTA'] / allAttempts * 100 if allAttempts > 0 else 0
        player_stats['AST/TOV'] = player_stats['AST'] / player_stats['TOV']
        player_stats['AST%'] = 100 * player_stats['AST'] / (((MP / (TmMP / 5)) * TmFGM) - player_stats['FGM'])
        player_stats['OREB%'] = 100 * (player_stats['OREB'] * (TmMP / 5)) / (MP * (TmORB + OppDRB))
        player_stats['DREB%'] = 100 * (player_stats['DREB'] * (TmMP / 5)) / (MP * (TmDRB + OppORB))
        player_stats['TREB%'] =  100 * (player_stats['TREB'] * (TmMP / 5)) / (MP * (TmTRB + OppTRB))
        player_stats['USG%'] = 100 * ((player_stats['FGA'] + 0.44 * player_stats['FTA'] + player_stats['TOV']) * (TmMP / 5)) / (MP * (TmFGA + 0.44 * TmFTA + TmTOV))
    except:
        pass
    return player_stats


In [49]:
def get_player_advanced_stats(player_id, all_games_data, total_team_stats):
    """
    Calculates advanced stats, using pre-calculated team totals if the
    player participated in all games.
    """
    player_games_dict = {
        match_id: game for match_id, game in all_games_data.items()
        if player_id in game.get('player_stats', {})
    }

    if not player_games_dict:
        return {"error": f"Player {player_id} did not play in any completed games."}

    # --- New Conditional Logic ---
    # Check if the player's game count matches the team's total game count.
    team_totals_for_player = dict()
    if len(player_games_dict) == len(all_games_data):
        # If yes, use the efficient pre-calculated total team stats.
        team_totals_for_player = total_team_stats
    else:
        # If no, calculate the team stats from only the games the player played.
        res = get_team_advanced_stats(player_id, all_games_data)
        team_totals_for_player = res["team_totals"]
    #print(team_totals_for_player)
    
    # --- The rest of the calculation remains the same ---
    player_totals = calculate_player_total_stats(player_id, player_games_dict)
    
    player_MP = player_totals["timeOnCourt"] / 60
    player_adv_stats = calculate_player_advanced_stats(player_id, player_MP, team_totals_for_player)

    return {"team_totals": team_totals_for_player, "player_totals": player_totals, "player_adv_stats": player_adv_stats}

## Get the data

In [50]:
women_team_name = {11374: "ROC-A", 11375: "ROC-WUG", 11376: "Japan", 11377: "South Korea", 11378: "Philippines", 11379: "Thailand"}

all_team_stats = dict()
all_player_stats = dict()

for team_id, team_name in women_team_name.items():
    all_team_stats[team_name] = {
        'adv_stats': {},
        'team_totals': {}
    }
    match_id = get_completed_match_id(team_id)
    if len(match_id) == 0:
        print(f"No completed matches for team {team_id}")
        continue
    print(f"Completed match IDs for team {team_id}: {match_id}")

    all_games_data = fetch_all_game_data(team_id, match_id)
    # with open('./json/all_games_data.json', 'w') as f:
    #     json.dump(all_games_data, f, indent=4)
    res = get_team_advanced_stats(None, all_games_data)
    team_adv_stats = res['adv_stats']
    total_team_stats = res['team_totals']

    
    all_team_stats[team_name]['adv_stats'] = team_adv_stats
    all_team_stats[team_name]['team_totals'] = total_team_stats

    # Calculate PLayer stats
    url = f"https://jonescup.meetagile.com/api/squad-team-squads/{team_id}"
    response = requests.get(url)
    data = response.json()

    for roster in data['rosters']:
        
        roster_id = roster['id']
        player_stats = get_player_advanced_stats(roster_id, all_games_data, total_team_stats)

        if "error" in player_stats:
            print(player_stats['error'])
            continue
        
        # store stats
        player_name = roster['personalInfo']['name']
        avatarUrl = roster['personalInfo']["avatarUrl"]
        all_player_stats[player_name] = {
            'squad_id': team_id,
            'roster_id': roster_id,
            'position': roster['position'],           
            'avatarUrl': avatarUrl,
            'adv_stats': player_stats['player_adv_stats'],
            "player_totals": player_stats['player_totals'],
            "team_totals": player_stats['team_totals']
        }
    
    

# # Write the advanced stats to a JSON file (tmp folder in docker GCP)
with open('./json/all_team_stats.json', 'w') as f:
    json.dump(all_team_stats, f, indent=4)

with open('./json/all_player_stats.json', 'w') as f:
    json.dump(all_player_stats, f, indent=4)



Completed match IDs for team 11374: [7972, 7969, 7966]
Player 154239 did not play in any completed games.
Completed match IDs for team 11375: [7969, 7965, 7971]
Completed match IDs for team 11376: [7967, 7964, 7971]
Player 154262 did not play in any completed games.
Completed match IDs for team 11377: [7967, 7965, 7970]
Completed match IDs for team 11378: [7968, 7966, 7970]
Player 154285 did not play in any completed games.
Completed match IDs for team 11379: [7972, 7964, 7968]


In [48]:
with open('./json/all_games_data.json', 'w') as f:
    json.dump(all_games_data, f, indent=4)


## Version 1

In [122]:
def create_accumalated_stat(team_id, match_id):
    team_stats = dict()
    player_MP = dict()
    team_stats["teamMinutesPlayed"] = dict()
    for id in match_id:
        url = f"https://jonescup.meetagile.com/api/match-basketball-stats/{id}"
        response = requests.get(url)
        data = response.json()

        squads = data['squads']
        for squad in squads:
            accu_stat = squad['stats']['accumulatedStats']
            # main team
            if squad['squadId'] == team_id:
                
                # FGA, FGM, 3PM, FTA, TO, OREB, score
                team_stats["fieldGoalsAttempted"] =  accu_stat['fieldGoalsAttempted'] + team_stats.get("fieldGoalsAttempted", 0)
                team_stats["fieldGoalsMade"] = accu_stat["fieldGoalsMade"] + team_stats.get("fieldGoalsMade", 0)
                team_stats["threePointersMade"] = accu_stat["threePointersMade"] + team_stats.get("threePointersMade", 0)
                team_stats["freeThrowsAttempted"] = accu_stat["freeThrowsAttempted"] + team_stats.get("freeThrowsAttempted", 0)
                team_stats["turnovers"] = accu_stat["turnovers"] + team_stats.get("turnovers", 0)
                team_stats["squadOffensiveRebounds"] = accu_stat["squadOffensiveRebounds"] + team_stats.get("squadOffensiveRebounds", 0)
                team_stats["squadDefensiveRebounds"] = accu_stat["squadDefensiveRebounds"] + team_stats.get("squadDefensiveRebounds", 0)
                team_stats["score"] = accu_stat["wonScore"] + team_stats.get("score", 0)
                team_stats["teamMinutesPlayed"][id] = (40 + ( len(data['rounds']) - 4 ) * 5) * 5 # 40 minutes for the first 4 rounds, and 5 minutes for overtime
                team_stats["teamMinutesPlayed"]['total'] = team_stats["teamMinutesPlayed"][id] + team_stats['teamMinutesPlayed'].get('total', 0)
                team_stats['possessions'] = team_stats['fieldGoalsAttempted'] + team_stats['turnovers'] + 0.44 * team_stats['freeThrowsAttempted'] - team_stats['squadOffensiveRebounds']
            # opponent team
            else:
                # FGA, FGM, 3PM, FTA, TO, OREB, DREB, score
                team_stats["Op-fieldGoalsAttempted"] =  accu_stat['fieldGoalsAttempted'] + team_stats.get("Op-fieldGoalsAttempted", 0)
                team_stats["Op-fieldGoalsMade"] = accu_stat["fieldGoalsMade"] + team_stats.get("Op-fieldGoalsMade", 0)
                team_stats["Op-threePointersMade"] = accu_stat["threePointersMade"] + team_stats.get("Op-threePointersMade", 0)
                team_stats["Op-freeThrowsAttempted"] = accu_stat["freeThrowsAttempted"] + team_stats.get("Op-freeThrowsAttempted", 0)
                team_stats["Op-turnovers"] = accu_stat["turnovers"] + team_stats.get("Op-turnovers", 0)
                team_stats["Op-squadOffensiveRebounds"] = accu_stat["squadOffensiveRebounds"] + team_stats.get("Op-squadOffensiveRebounds", 0)
                team_stats["Op-squadDefensiveRebounds"] = accu_stat["squadDefensiveRebounds"] + team_stats.get("Op-squadDefensiveRebounds", 0)
                team_stats["Op-score"] = accu_stat["wonScore"] + team_stats.get("Op-score", 0)
                team_stats['Op-possessions'] = team_stats['Op-fieldGoalsAttempted'] + team_stats['Op-turnovers'] + 0.44 * team_stats['Op-freeThrowsAttempted'] - team_stats['Op-squadOffensiveRebounds']
        
        rosters = data['rosters']
        for roster in rosters:
            if roster['squadId'] == team_id:
                rosterId = roster['rosterId']
                # Initialize if not already
                if rosterId not in player_MP:
                    player_MP[rosterId] = dict()
                player_MP[rosterId][id] = roster['stats']['accumulatedStats']['timeOnCourt']
                player_MP[rosterId]['total'] = player_MP[rosterId][id] + player_MP[rosterId].get('total', 0)
    
    return team_stats, player_MP



                


    
    

In [ ]:
women_team_name = {11374: "ROC-A", 11375: "ROC-WUG", 11376: "Japan", 11377: "South Korea", 11378: "Philippines", 11379: "Thailand"}
all_team_adv_stats = dict()
player_MP_stats = dict()
all_team_stats = dict()

for team_id, team_name in women_team_name.items():
    all_team_adv_stats[team_name] = dict()
    match_id = get_completed_match_id(team_id)
    if len(match_id) == 0:
        print(f"No completed matches for team {team_id}")
        continue
    print(f"Completed match IDs for team {team_id}: {match_id}")

    ## Get accumalated stats for team id
    accumated_stat, player_MP = create_accumalated_stat(team_id, match_id)
    player_MP_stats[team_id] = player_MP
    all_team_stats[team_id] = accumated_stat
    ## Calculate OFR, DFR, Possession, Pace, Four Factors
    team_stats = calculate_advanced_stats(accumated_stat)
    team_stats['squadId'] = team_id
    all_team_adv_stats[team_name] = team_stats
    print(f"Advanced stats for team {team_id}: {team_stats}")

# Write the advanced stats to a JSON file (tmp folder in docker GCP)
with open('./json/all_team_stats.json', 'w') as f:
    json.dump(all_team_adv_stats, f, indent=4)



## Player Data

## Roster infomation of specifc team id
https://jonescup.meetagile.com/api/squad-team-squads/{team_id}

## Player Stats
https://jonescup.meetagile.com/api/roster-division-basketball-stats/{roster_id}
if return = [] => no need to consider

Get accumalated Data => 'averageStats' * "matchCount"
## No
- OFFRTG
- DEFRTG
- NETRTG = OFFRTG - DEFRTG
- PACE
- POSS
- PPP(Points Per Possession) = Points ÷ (Field Goal Attempts + (0.44 x Free Throw Attempts) + Turnovers)
- TO RATIO
- AST RATIO = (AST ÷ (FGA + (0.44 x FTA) + AST + TO)) x 100%

## Yes
### Advance Data
- 2PA% = 2PA / (2PA + 3PA + FTA) DONE
- 3PA% = 3PA / (2PA + 3PA + FTA) DONE
- FTA% = FTA / (2PA + 3PA + FTA) DONE

- AST% = 100 * AST / (((MP / (Tm MP / 5)) * Tm FGM) - FGM) DONE
- AST/TO = AST / TO DONE

- OREB% = 100 * (ORB * (Tm MP / 5)) / (MP * (Tm ORB + Opp DRB)) DONE
- DREB% = 100 * (DRB * (Tm MP / 5)) / (MP * (Tm DRB + Opp ORB)) DONE
- TREB% =  100 * (TRB * (Tm MP / 5)) / (MP * (Tm TRB + Opp TRB)) DONE

- EFG% DONE
- TS% DONE
- USG% = 100 * ((FGA + 0.44 * FTA + TOV) * (Tm MP / 5)) / (MP * (Tm FGA + 0.44 * Tm FTA + Tm TOV))

### Basic Data (avergae) per game
- Points (average) DONE
- Assist (average) DONE
- Rebound (average) DONE
- steal (average) DONE
- blocks (average) DONE
- 2P percentage (acc) need to have attemp
- 3P percentage (acc) DONE need to have attemp
- Free throw percentage (acc) DONE need to have attemp
- Field Goal Percentage (acc) DONE
 



## 根據roster_id 有上的那幾場加總

TmTOV = team_stats['turnovers']

    TmMP = team_stats['teamMinutesPlayed']

    TmFGM = team_stats['fieldGoalsMade']

    TmFGA = team_stats['fieldGoalsAttempted']
    
    TmFTA = team_stats['freeThrowsAttempted']
    
    TmORB = team_stats['squadOffensiveRebounds']
    TmDRB = team_stats['squadDefensiveRebounds']
    TmTRB = TmORB + TmDRB

    OppORB = team_stats['Op-squadOffensiveRebounds']
    OppDRB = team_stats['Op-squadDefensiveRebounds']
    OppTRB = OppORB + OppDRB

多場數據怎麼算？

player_stats['OREB%'] = 100 * (player_stats['OREB'] * (TmMP / 5)) / (MP * (TmORB + OppDRB))

player_stats['DREB%'] = 100 * (player_stats['DREB'] * (TmMP / 5)) / (MP * (TmDRB + OppORB))

player_stats['TREB%'] =  100 * (player_stats['TREB'] * (TmMP / 5)) / (MP * (TmTRB + OppTRB))
    

In [ ]:
def get_player_stats(team_id, roster_id):
    player_stats = dict()
    url = f"https://jonescup.meetagile.com/api/roster-division-basketball-stats/{roster_id}"
    response = requests.get(url)
    data = response.json()

    # # not play yet
    # if len(data) == 0:
    #     return player_stats
    
    # team_stats = all_team_stats[team_id]
    # MP = player_MP_stats[team_id][roster_id] / 60
    # TmTOV = team_stats['turnovers']
    # TmMP = team_stats['teamMinutesPlayed']
    # TmFGM = team_stats['fieldGoalsMade']
    # TmFGA = team_stats['fieldGoalsAttempted']
    # TmFTA = team_stats['freeThrowsAttempted']
    
    # TmORB = team_stats['squadOffensiveRebounds']
    # TmDRB = team_stats['squadDefensiveRebounds']
    # TmTRB = TmORB + TmDRB

    # OppORB = team_stats['Op-squadOffensiveRebounds']
    # OppDRB = team_stats['Op-squadDefensiveRebounds']
    # OppTRB = OppORB + OppDRB

    stats = data[0]['stats']
    matchCount = stats['matchCount']
    average_stats = stats['averageStats']
    percentage_stats = stats['percentageStats']

    player_stats['GP'] = stats['matchCount']
    player_stats['average-points'] = average_stats['score']
    player_stats['average-assists'] = average_stats['assists']
    player_stats['average-rebounds'] = average_stats['rebounds']
    player_stats['average-steals'] = average_stats['steals']
    player_stats['average-blocks'] = average_stats['blocks']
    player_stats['average-turnovers'] = average_stats['turnovers']

    player_stats['3P%'] = percentage_stats['threePointersPercentage'] * 100
    player_stats['FT%'] = percentage_stats['freeThrowsPercentage'] * 100
    player_stats['FG%'] = percentage_stats['fieldGoalsPercentage'] * 100
    player_stats['eFG%'] = percentage_stats['effectiveFieldGoalsPercentage'] * 100
    player_stats['TS%'] = percentage_stats['trueShootingPercentage'] * 100

    player_stats['2PA'] = average_stats['twoPointersAttempted'] * matchCount
    player_stats['2PM'] = average_stats['twoPointersMade'] * matchCount
    player_stats['twoPointersPercentage'] = player_stats['2PM'] / player_stats['2PA'] * 100 if player_stats['2PA'] != 0 else 0

    player_stats['3PA'] = average_stats['threePointersAttempted'] * matchCount
    player_stats['FTA'] = average_stats['freeThrowsAttempted'] * matchCount
    player_stats['AST'] = average_stats['assists'] * matchCount
    player_stats['FGM'] = average_stats['fieldGoalsMade'] * matchCount
    player_stats['FGA'] = average_stats['fieldGoalsAttempted'] * matchCount
    player_stats['TOV'] = average_stats['turnovers'] * matchCount
    player_stats['OREB'] = average_stats['offensiveRebounds'] * matchCount
    player_stats['DREB'] = average_stats['defensiveRebounds'] * matchCount
    player_stats['TREB'] = player_stats['OREB'] + player_stats['DREB']

    player_stats['2PA%'] = player_stats['2PA'] / (player_stats['2PA'] + player_stats['3PA'] + player_stats['FTA']) * 100
    player_stats['3PA%'] = player_stats['3PA'] / (player_stats['2PA'] + player_stats['3PA'] + player_stats['FTA']) * 100
    player_stats['FTA%'] = player_stats['FTA'] / (player_stats['2PA'] + player_stats['3PA'] + player_stats['FTA']) * 100
    player_stats['AST/TOV'] = player_stats['AST'] / player_stats['TOV']
    player_stats['AST%'] = 100 * player_stats['AST'] / (((MP / (TmMP / 5)) * TmFGM) - player_stats['FGM'])
    player_stats['OREB%'] = 100 * (player_stats['OREB'] * (TmMP / 5)) / (MP * (TmORB + OppDRB))
    player_stats['DREB%'] = 100 * (player_stats['DREB'] * (TmMP / 5)) / (MP * (TmDRB + OppORB))
    player_stats['TREB%'] =  100 * (player_stats['TREB'] * (TmMP / 5)) / (MP * (TmTRB + OppTRB))
    player_stats['USG%'] = 100 * ((player_stats['FGA'] + 0.44 * player_stats['FTA'] + player_stats['TOV']) * (TmMP / 5)) / (MP * (TmFGA + 0.44 * TmFTA + TmTOV))

    return player_stats



In [82]:
get_player_stats(11377, 154277)

{'GP': 2,
 'average-points': 4,
 'average-assists': 2,
 'average-rebounds': 3.5,
 'average-steals': 0,
 'average-blocks': 0.5,
 'average-turnovers': 2,
 '3P%': 0,
 'FT%': 100,
 'FG%': 30.0,
 'eFG%': 30.0,
 'TS%': 36.764705882352935,
 '2PA': 4,
 '2PM': 3.0,
 'twoPointersPercentage': 75.0,
 '3PA': 6,
 'FTA': 2,
 'AST': 4,
 'FGM': 3.0,
 'FGA': 10,
 'TOV': 4,
 'OREB': 0,
 'DREB': 7.0,
 'TREB': 7.0,
 '2PA%': 33.33333333333333,
 '3PA%': 50.0,
 'FTA%': 16.666666666666664,
 'AST/TOV': 1.0,
 'AST%': 14.298906729422978,
 'OREB%': 0.0,
 'DREB%': 175.14595496246872,
 'TREB%': 63.68943816817045,
 'USG%': 16.110353031103017}

In [ ]:
def get_team_players_stat(team_id):
    team_player_stats = []
    url = f"https://jonescup.meetagile.com/api/squad-team-squads/{team_id}"
    response = requests.get(url)
    data = response.json()

    rosters = data["rosters"]
    for roster in rosters:
        ps = roster
        roster_id = roster["id"]
        player_stats = get_player_stats(roster_id)
        ps['stats'] = player_stats

        team_player_stats.append(ps)


        